In [ ]:
%matplotlib widget

from pathlib import Path
from dataclasses import dataclass

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import h5py
import pickle
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display,clear_output
import ncempy.algo as nalgo
import ncempy.io as nio

In [ ]:
# Input File Path (Change according to your computer):
emd_file = Path(r'C:\Users\stoop\Downloads\openNCEM-master\openNCEM-master\ncempy\data\20220505_HfO2_19.emd')

In [ ]:


def import_data_metadata(emd_file, index, data_name='images2048'):
    with h5py.File(emd_file, 'r') as f:
        pickledict = f[f'data/{data_name}/pickledict'][index] # take each images
        d = pickle.loads(pickledict, encoding='bytes')
        data = f[f'data/{data_name}/data'][index]
    metadata = {}
    metadata['pixel_size'] = d['_calX'] # this assumes X and Y have the same pixel size 
    metadata['brightness'] = d['_brightness'] # HAADF brightness 
    metadata['contrast'] = d['_contrast'] # HAADF contrast
    metadata['comment'] = d['_comment'] # image comment
    metadata['camera_length'] = d['_cameralength'] # camera length label
    metadata['camera_length_index'] = d['_cameralengthindex'] # camera length index
    metadata['convergence_angle'] = d['_convergenceAngle'] # in radians
    metadata['defocus'] = d['_defocus']
    metadata['dwell_time'] = d['_dwell']
    metadata['accelerating_voltage'] = d['_HTValue']
    metadata['image_shape'] = d['_imageDataShape']
    metadata['magnification'] = d['_mag']
    metadata['stage_position'] = d['_position'] # X, Y, Z, Alpha, Gamma
    metadata['scan_rotation'] = d['_rot'] # STEM scan rotation
    
    return data, d, metadata

def load_all_images(emd_file, data_name='images2048'):
    with h5py.File(emd_file, 'r') as f:
        data = f[f'data/{data_name}/data'][:]
    return data

In [ ]:
emd = nio.emd.emdReader(emd_file)
index = 0
data_name = 'images2048' # options are 'images2048', 'images1024', 'images512', 'images256', 'flucam'
data, d, metadata = import_data_metadata(emd_file, index, data_name = data_name)

In [ ]:
with h5py.File(emd_file, 'r') as f0:
    length = f0['data/images2048/data'].shape[0]
    
ref_image, ref_dict, ref_metadata = import_data_metadata(emd_file, 0, data_name='images1024')
final_image, final_dict, final_metadata = import_data_metadata(emd_file, 1, data_name='images1024')
HM_image, HM_dict, HM_metadata = import_data_metadata(emd_file, 0, data_name='images2048')


pos = []
for ii in range(length):
    final_image, final_dict, final_metadata = import_data_metadata(emd_file, ii+1, data_name='images1024')
    _, __ = nalgo.align.image_correlate(final_image-final_image.mean(), ref_image-ref_image.mean())
#     print(__)+
    pos.append(__)
pos = np.array(pos)

sh = ref_image.shape[0]//2 - 1

fg1,(ax0, ax1, ax2) = plt.subplots(1,3,figsize=(12,4))

im1 = ax1.imshow(final_image)
p1 = ax0.plot(4,4,'or')
p1 = p1[0]
ax0.imshow(ref_image)
im2 = ax2.imshow(HM_image)
ax0.set(title='Reference Image:')


def get_ims(Image):
    HM_image, HM_dict, HM_metadata = import_data_metadata(emd_file, Image, data_name='images2048')
    LM_image, LM_dict, LM_metadata = import_data_metadata(emd_file, Image +1, data_name='images1024')
    x = sh+pos[Image,1]
    y = sh+pos[Image,0]
    p1.set_xdata(x)
    p1.set_ydata(y)
    #im0.set_data(ref_image)
    im1.set_data(LM_image)
    im2.set_data(HM_image)


ax1.set(title=f'Zoomed Out Image:')
ax2.set(title=f'High Mag Image:')   
    
  
ax0.axis('off')
ax1.axis('off')
ax2.axis('off')


w = interactive(get_ims, Image = range(0,length))
display(w)


